In [3]:
import numpy as np
import math
from sklearn import datasets
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [93]:
class WeakTree(DecisionTreeClassifier):
    def __init__(self,max_depth =1):
        super().__init__(max_depth = max_depth)
        self.polarity = 1
        self.alpha = None
        

In [122]:
class Adaboost1():
    def __init__(self, n_clf = 5):
        self.n_clf = n_clf
        
    def fit(self,X,y):
        n_samples, n_features = np.shape(X)
        w = np.full(n_samples,1/(n_samples))
        self.clfs = []
        for _ in range(self.n_clf):
            clf = WeakTree(max_depth = 1)
            clf.fit(X,y,sample_weight = w)
            error =1- clf.score(X,y,w)
            clf.alpha = 0.5*math.log((1.-error)/(error+1e-10))
            pred_y = clf.predict(X)
            cc = (pred_y == y)*1.0
            cc[cc==0] = -1.0
            w = w*np.exp(-cc*clf.alpha)
            w /=np.sum(w)
            self.clfs.append(clf)
        
    def predict(self,X):
        y_pred = np.zeros((X.shape[0],))
        for tree in self.clfs:
            y_hat = tree.predict(X)
            y_hat[y_hat == 0] =-1.
            y_pred += tree.alpha*y_hat
        
        return np.sign(y_pred)
    
    def accuracy(self,X,y):
        y_ = y.copy()
        y_[y_ ==0] = -1.
        
        y_pred= self.predict(X)
        return (y_ == y_pred).mean()
                    
                

In [118]:
np.random.seed(1)
K =20
p =2
N =300
mu = np.random.normal(0.,4.,(K,2))
component = np.random.randint(0,K,(N,))
assignment = np.random.randint(0,2,(K,))
X = mu[component,:] + np.random.normal(0.,1.,(N,p))
y = assignment[component]
        
            

In [103]:
        n_samples, n_features = np.shape(X)
        w = np.full(n_samples,1/(n_samples))
        clfs = []
        for _ in range(10):
            clf = WeakTree(max_depth = 1)
            clf.fit(X,y,sample_weight = w)
            error =1- clf.score(X,y,w)
            clf.alpha = 0.5*math.log((1.-error)/(error+1e-10))
            pred_y = clf.predict(X)
            cc = (pred_y == y)*1.0
            cc[cc==0] = -1.0
            w = w*np.exp(-cc*clf.alpha)
            w /=np.sum(w)
            clfs.append(clf)

In [104]:
        y_pred = np.zeros((X.shape[0],))
        for tree in clfs:
            y_hat = tree.predict(X)
            y_hat[y_hat ==0] = -1.
            y_pred += tree.alpha*y_hat

In [105]:
sign = np.sign(y_pred)

In [106]:
y[y == 0] = -1.


In [107]:
(y ==sign).mean()

0.82

In [127]:
ad1 = Adaboost1(20)
ad1.fit(X,y)
ad1.accuracy(X,y)

0.83

In [128]:
ad = Adaboost(20)
ad.fit(X,y_)
(ad.predict(X) ==y_).mean()

0.8533333333333334

In [129]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=20)
clf.fit(X, y)
(clf.predict(X) ==y).mean()

0.88

In [115]:
class DecisionStump():
    def __init__(self):
        # Determines if sample shall be classified as -1 or 1 given threshold
        self.polarity = 1
        # The index of the feature used to make classification
        self.feature_index = None
        # The threshold value that the feature should be measured against
        self.threshold = None
        # Value indicative of the classifier's accuracy
        self.alpha = None

class Adaboost():
    """Boosting method that uses a number of weak classifiers in 
    ensemble to make a strong classifier. This implementation uses decision
    stumps, which is a one level Decision Tree. 
    Parameters:
    -----------
    n_clf: int
        The number of weak classifiers that will be used. 
    """
    def __init__(self, n_clf=5):
        self.n_clf = n_clf

    def fit(self, X, y):
        n_samples, n_features = np.shape(X)

        # Initialize weights to 1/N
        w = np.full(n_samples, (1 / n_samples))
        
        self.clfs = []
        # Iterate through classifiers
        for _ in range(self.n_clf):
            clf = DecisionStump()
            # Minimum error given for using a certain feature value threshold
            # for predicting sample label
            min_error = float('inf')
            # Iterate throught every unique feature value and see what value
            # makes the best threshold for predicting y
            for feature_i in range(n_features):
                feature_values = np.expand_dims(X[:, feature_i], axis=1)
                unique_values = np.unique(feature_values)
                # Try every unique feature value as threshold
                for threshold in unique_values:
                    p = 1
                    # Set all predictions to '1' initially
                    prediction = np.ones(np.shape(y))
                    # Label the samples whose values are below threshold as '-1'
                    prediction[X[:, feature_i] < threshold] = -1
                    # Error = sum of weights of misclassified samples
                    error = sum(w[y != prediction])
                    
                    # If the error is over 50% we flip the polarity so that samples that
                    # were classified as 0 are classified as 1, and vice versa
                    # E.g error = 0.8 => (1 - error) = 0.2
                    if error > 0.5:
                        error = 1 - error
                        p = -1

                    # If this threshold resulted in the smallest error we save the
                    # configuration
                    if error < min_error:
                        clf.polarity = p
                        clf.threshold = threshold
                        clf.feature_index = feature_i
                        min_error = error
            # Calculate the alpha which is used to update the sample weights,
            # Alpha is also an approximation of this classifier's proficiency
            clf.alpha = 0.5 * math.log((1.0 - min_error) / (min_error + 1e-10))
            # Set all predictions to '1' initially
            predictions = np.ones(np.shape(y))
            # The indexes where the sample values are below threshold
            negative_idx = (clf.polarity * X[:, clf.feature_index] < clf.polarity * clf.threshold)
            # Label those as '-1'
            predictions[negative_idx] = -1
            # Calculate new weights 
            # Missclassified samples gets larger weights and correctly classified samples smaller
            w *= np.exp(-clf.alpha * y * predictions)
            # Normalize to one
            w /= np.sum(w)

            # Save classifier
            self.clfs.append(clf)

    def predict(self, X):
        n_samples = np.shape(X)[0]
        y_pred = np.zeros((n_samples, 1))
        # For each classifier => label the samples
        for clf in self.clfs:
            # Set all predictions to '1' initially
            predictions = np.ones(np.shape(y_pred))
            # The indexes where the sample values are below threshold
            negative_idx = (clf.polarity * X[:, clf.feature_index] < clf.polarity * clf.threshold)
            # Label those as '-1'
            predictions[negative_idx] = -1
            # Add predictions weighted by the classifiers alpha
            # (alpha indicative of classifier's proficiency)
            y_pred += clf.alpha * predictions

        # Return sign of prediction sum
        y_pred = np.sign(y_pred).flatten()

        return y_pred